# GENESIS Level 0 Training

1. **Runtime → Change runtime type → GPU (T4)**
2. Run all cells in order

In [70]:
# Step 1: Setup environment
import sys
import os
import subprocess

# Clear any cached modules
mods_to_remove = [k for k in sys.modules if 'genesis' in k or 'core' in k or 'levels' in k]
for mod in mods_to_remove:
    del sys.modules[mod]

# Clone fresh
if os.path.exists('genesis'):
    try:
        subprocess.run(['rm', '-rf', 'genesis'], check=True)
    except subprocess.CalledProcessError as e:
        print(f"Error removing existing genesis directory: {e}")
        sys.exit(1)

try:
    subprocess.run(['git', 'clone', 'https://github.com/0xMayoor/genesis.git'], check=True)
except subprocess.CalledProcessError as e:
    print(f"Error cloning repository: {e}")
    sys.exit(1)

os.chdir('genesis')

# Install
try:
    subprocess.run([sys.executable, '-m', 'pip', 'install', '-q', 'torch', 'transformers', 'peft', 'accelerate', 'capstone', 'hypothesis'], check=True)
    subprocess.run([sys.executable, '-m', 'pip', 'install', '-e', '.', '--force-reinstall', '--no-deps', '-q'], check=True)
except subprocess.CalledProcessError as e:
    print(f"Error installing dependencies: {e}")
    sys.exit(1)


# Add to path
sys.path.insert(0, os.getcwd())

print(f"✅ Setup complete. Working dir: {os.getcwd()}")

✅ Setup complete. Working dir: /content/genesis/genesis/genesis/genesis/genesis/genesis/genesis/genesis/genesis/genesis


In [71]:
# Step 2: Check GPU
import torch
print(f"CUDA: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")

CUDA: True
GPU: Tesla T4


In [72]:
# Step 3: Train
from pathlib import Path
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import LoraConfig, get_peft_model, TaskType
from torch.utils.data import DataLoader, Dataset
import json

# Config
MODEL_NAME = "distilgpt2"
BATCH_SIZE = 8
EPOCHS = 3
LR = 2e-5
MAX_LENGTH = 256
DATASET_PATH = Path("genesis_datasets/level0/train.jsonl")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Device: {device}")

# Load tokenizer and model
print("Loading model...")
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
tokenizer.pad_token = tokenizer.eos_token

model = AutoModelForCausalLM.from_pretrained(MODEL_NAME, torch_dtype=torch.float16)

# Apply LoRA
print("Applying LoRA...")
lora_config = LoraConfig(
    task_type=TaskType.CAUSAL_LM,
    r=8,
    lora_alpha=16,
    lora_dropout=0.1,
    target_modules=["c_attn", "c_proj"],
)
model = get_peft_model(model, lora_config)
model.enable_input_require_grads()  # Critical for gradient flow
model.print_trainable_parameters()
model.to(device)

# Dataset
class Level0Dataset(Dataset):
    def __init__(self, path, tokenizer, max_length):
        self.samples = []
        with open(path) as f:
            for line in f:
                self.samples.append(json.loads(line))
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        sample = self.samples[idx]
        # Format: bytes -> expected output
        hex_bytes = sample.get("raw_bytes", "")
        expected = sample.get("expected_mnemonic", "unknown")
        text = f"Disassemble: {hex_bytes}\nOutput: {expected}"

        encoded = self.tokenizer(
            text,
            truncation=True,
            max_length=self.max_length,
            padding="max_length",
            return_tensors="pt"
        )
        return {
            "input_ids": encoded["input_ids"].squeeze(),
            "attention_mask": encoded["attention_mask"].squeeze(),
            "labels": encoded["input_ids"].squeeze(),
        }

print(f"Loading dataset from {DATASET_PATH}...")
dataset = Level0Dataset(DATASET_PATH, tokenizer, MAX_LENGTH)
dataloader = DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=True)
print(f"Dataset: {len(dataset)} samples, {len(dataloader)} batches")

# Training
optimizer = torch.optim.AdamW(model.parameters(), lr=LR)
model.train()

print(f"\nStarting training for {EPOCHS} epochs...")
print("="*50)

for epoch in range(EPOCHS):
    total_loss = 0
    for batch_idx, batch in enumerate(dataloader):
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["labels"].to(device)

        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

        if batch_idx % 50 == 0:
            print(f"Epoch {epoch+1}/{EPOCHS} | Batch {batch_idx}/{len(dataloader)} | Loss: {loss.item():.4f}")

    avg_loss = total_loss / len(dataloader)
    print(f"Epoch {epoch+1} complete. Avg Loss: {avg_loss:.4f}")

print("="*50)
print("🎉 Training Complete!")

Device: cuda
Loading model...
Applying LoRA...


/usr/local/lib/python3.12/dist-packages/peft/tuners/lora/layer.py:2285: UserWarning: fan_in_fan_out is set to False but the target module is `Conv1D`. Setting fan_in_fan_out to True.
  warnings.warn(


trainable params: 405,504 || all params: 82,318,080 || trainable%: 0.4926
Loading dataset from genesis_datasets/level0/train.jsonl...
Dataset: 1493 samples, 187 batches

Starting training for 3 epochs...
Epoch 1/3 | Batch 0/187 | Loss: 9.7177
Epoch 1/3 | Batch 50/187 | Loss: 8.3004
Epoch 1/3 | Batch 100/187 | Loss: 3.1706
Epoch 1/3 | Batch 150/187 | Loss: 0.7163
Epoch 1 complete. Avg Loss: 4.5266
Epoch 2/3 | Batch 0/187 | Loss: 0.6649
Epoch 2/3 | Batch 50/187 | Loss: 0.5545
Epoch 2/3 | Batch 100/187 | Loss: 0.6483
Epoch 2/3 | Batch 150/187 | Loss: 0.4817
Epoch 2 complete. Avg Loss: 0.5327
Epoch 3/3 | Batch 0/187 | Loss: 0.3305
Epoch 3/3 | Batch 50/187 | Loss: 0.5309
Epoch 3/3 | Batch 100/187 | Loss: 0.6897
Epoch 3/3 | Batch 150/187 | Loss: 0.5133
Epoch 3 complete. Avg Loss: 0.5501
🎉 Training Complete!


In [73]:
# Step 4: Save model
save_path = Path("models/level0")
save_path.mkdir(parents=True, exist_ok=True)
model.save_pretrained(save_path)
tokenizer.save_pretrained(save_path)
print(f"✅ Model saved to {save_path}")

✅ Model saved to models/level0


In [75]:
# Step 5: Download
import subprocess
from google.colab import files

try:
    subprocess.run(['zip', '-r', 'level0_model.zip', 'models/level0/'], check=True)
    files.download('level0_model.zip')
    print("📦 Download started!")
except subprocess.CalledProcessError as e:
    print(f"Error zipping model: {e}")
except Exception as e:
    print(f"Error during download: {e}")

Error during download: 'NoneType' object has no attribute 'kernel'
